In [776]:
import math

class Dual:
    def __init__(self, value=0., derivative=0.):
        self.value = value
        self.derivative = derivative
        
    def __add__(self, other):
        if isinstance(other, self.__class__):
            return Dual(self.value + other.value, self.derivative + other.derivative)
        else:
            return self + Dual(other)
        
    __radd__ = __add__
    
    def __sub__(self, other):
        return self + -other
    
    def __rsub__(self, other):
        return other + -self
    
    def __mul__(self, other):
        if isinstance(other, self.__class__):
            return Dual(self.value * other.value, self.value*other.derivative + self.derivative * other.value)
        else:
            return self * Dual(other)
        
    __rmul__ = __mul__
    
    def __truediv__(self, other):
        return self * other**(-1.)
    
    def __rtruediv__(self, other):
        return other * self**(-1.)
    
    def __pow__(self, other):
        if isinstance(other, self.__class__):
            return Dual(self.value**other.value, self.derivative * other.value * self.value**(other.value-1))
        else:
            return Dual(self.value**other, self.derivative * other * self.value**(other-1))
        
    def __rpow__(self, other):
        if isinstance(other, self.__class__):
            return Dual(other.value**self.value, self.derivative * other.value**self.value * math.log(other.value))
        else:
            return Dual(other**self.value, self.derivative * other**self.value * math.log(other))
        
    def __neg__(self):
        return -1. * self
    
    def __repr__(self):
        return '< Dual value: {}, derivative: {} >'.format(self.value, self.derivative)
    
    @staticmethod
    def create_diff_fn(fn):
        def diff_fn(*argv):
            # return a list of functions for each argument
            Jacobian = []
            Dual_arguments = [Dual(x) for x in argv]
            for input_arg in Dual_arguments:
                input_arg.derivative = 1.
                result = fn(*Dual_arguments)
                Jacobian.append(result.derivative)
                input_arg.derivative = 0.

            return Jacobian
        
        return diff_fn
    
    
class Variable:
    def __init__(self, operation, input_variables=[]):
        self.value = operation
        self.input_variables = input_variables
        self.gradient = 0.
        
    def calc_input_values(self):
        return [v.value for v in self.input_variables]
    
    def forward(self):                
        return self.forward_op(*self.calc_input_values())
        
    def backward(self, output_gradient=1.):
        self.gradient += output_gradient
        local_gradient = self.derivative_op(*self.calc_input_values())
        for differential, input_variable in zip(local_gradient, self.input_variables):
            input_variable.backward(differential * output_gradient)
            
    def clear_gradient(self):
        self.gradient = 0.
        for input_variable in self.input_variables:
            input_variable.clear_gradient()
    
    @property
    def value(self):
        return self.forward()
    
    @value.setter
    def value(self, value):
        if callable(value):
            self.forward_op = value
            
        else:
            self.forward_op = lambda : value
        
        self.derivative_op = Dual.create_diff_fn(self.forward_op)
    
    def __gt__(self, other):
        if isinstance(other, self.__class__):
            return self.value > other.value
        else:
            return self.value > other
        
    def __lt__(self, other):
        if isinstance(other, self.__class__):
            return self.value < other.value
        else:
            return self.value < other
        
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.forward() == other.forward()
        else:
            return self.forward() == other
        
    def __le__(self, other):
        return self == other or self < other
    
    def __ge__(self, other):
        return self == other or self > other
            
    def __add__(self, other):
        if isinstance(other, self.__class__):
            return Variable(lambda a,b : a+b, (self, other))
        else:
            return Variable(lambda a,b : a+b, (self, Variable(other)))
        
    __radd__ = __add__
    
    def __sub__(self, other):
        return self + -other
        
    def __rsub__(self, other):
        return other + -self
        
    
    def __mul__(self, other):
        if isinstance(other, self.__class__):
            return Variable(lambda a,b : a*b, (self, other))
        else:
            return Variable(lambda a,b : a*b, (self, Variable(other)))
    
    __rmul__ = __mul__
    
    def __truediv__(self, other):
        return self * other**(-1.)
        
    def __rtruediv__(self, other):
        return other * self**(-1.)
    
    def __pow__(self, other):
        if isinstance(other, self.__class__):
            return Variable(lambda a,b : a**b, (self, other))
        else:
            return Variable(lambda a,b : a**b, (self, Variable(other)))
        
    # call right version of power if dual
    # this is a tricky case to handle
    # instead consider universal a**b with exp(b*log(a)) (check sign of a)?
        
    def __rpow__(self, other):
        if isinstance(other, self.__class__):
            return Variable(lambda a,b : b.__rpow__(a) if isinstance(a, Dual) else a**b, (other, self))
        else:
            return Variable(lambda a,b : b.__rpow__(a) if isinstance(a, Dual) else a**b, (Variable(other), self))
        
    def __neg__(self):
        return -1.*self
    
    def __repr__(self):
        return '< Variable value: {}, gradient: {} >'.format(self.value, self.gradient)
    


def sin_series(x):
    # force copy
    if isinstance(x, Dual):
        result = Dual(x.value, x.derivative)
    else:
        result = x
    
    for i in range(40):
        factor = (-1)**(i+1)
        coeff = (i+1)*2 + 1
        result += factor * x**coeff / math.factorial(coeff)
                  
    return result
        
def sin(x):
    if isinstance(x, Dual):
        return Dual(math.sin(x.value), x.derivative * math.cos(x.value))
    if isinstance(x, Variable):
        return Variable(lambda a : sin(a), [x])
    else:
        return math.sin(x)

In [777]:
# tests

def super_complicated_function(x):
    return x * sin(x+6.)**2. / 2. - 2. / 2.**x

# by running the above in mathematica (wolfram alpha)
# direct link: https://www.wolframalpha.com/input/?i=derivative+x+*+sin%28x%2B6.%29**2.+%2F+2.+-+2%2F2**x
def d_super_complicated_function(x):
    return 2**(1 - x) * math.log(2) + 0.5 * sin(x + 6)**2 + x * math.sin(x + 6) * math.cos(x + 6)

for x in [-1., 0.5, 3., 2.]:
    print('Function Input: {}'.format(x))
    print('Function Value: {}'.format(super_complicated_function(x)))
    print('Function Symbolic Derivative: {}'.format(d_super_complicated_function(x)))
    print(super_complicated_function(Dual(x, 1.)))
    print('-'*32)


print('TESTING Variable')

for x in [-1., 0.5, 3., 2.]:
    print('Function Input: {}'.format(x))
    print('Function Value: {}'.format(super_complicated_function(x)))
    print('Function Symbolic Derivative: {}'.format(d_super_complicated_function(x)))
    x_v = Variable(x)
    L = super_complicated_function(x_v)
    print('Variable Function Output Value: {}'.format(L))
    L.backward()
    print('Input value: {}'.format(x_v))
    print('-'*32)


    

Function Input: -1.0
Function Value: -4.459767882269113
Function Symbolic Derivative: 3.504367159953579
< Dual value: -4.459767882269113, derivative: 3.504367159953579 >
--------------------------------
Function Input: 0.5
Function Value: -1.4026444100543694
Function Symbolic Derivative: 1.1084382073126584
< Dual value: -1.4026444100543694, derivative: 1.1084382073126582 >
--------------------------------
Function Input: 3.0
Function Value: 0.004762468816939924
Function Symbolic Derivative: -0.8682732520785479
< Dual value: 0.004762468816939924, derivative: -0.8682732520785477 >
--------------------------------
Function Input: 2.0
Function Value: 0.47882974016169233
Function Symbolic Derivative: 0.5480851436957535
< Dual value: 0.47882974016169233, derivative: 0.5480851436957535 >
--------------------------------
TESTING Variable
Function Input: -1.0
Function Value: -4.459767882269113
Function Symbolic Derivative: 3.504367159953579
Variable Function Output Value: < Variable value: -4.4

In [807]:
x = Variable(3.)

S = x * 0.17

Q = S / 2.

R = Q - 0.25

R.backward()

print(x)

< Variable value: 3.0, gradient: 0.085 >


In [762]:
# compose a graph of nodes

x = Variable(2.)

L = x * sin(x+6.)**2. / 2. - 2. / 2.**x

print(L)

L.backward()

print(x)

< Variable value: 0.47882974016169233, gradient: 0.0, operation: <function Variable.__add__.<locals>.<lambda> at 0x7f4d6ad011e0> >
< Variable value: 2.0, gradient: 0.5480851436957535, operation: <function Variable.value.<locals>.<lambda> at 0x7f4d6ad5f730> >


In [810]:
def forward_fn(x):
    for n in range(5):
        if n % 2 == 0:
            x = 3.*x
        else:
            x = x**(1./n) + 1./n
        
    return x

x = Variable(2.)
y = forward_fn(x)
print(y)

y.backward()
print(x)

< Variable value: 9.276772529143361, gradient: 0.0 >
< Variable value: 2.0, gradient: 1.1823960755919083 >


In [831]:
def fn(x):
    return x**2 + 0.2*(x-2)**4 + 2*x**3

# initialization
x = Variable(2.)
print('---- Initial Value ----')
print('fn(x): {}'.format(L))
print('x: {}'.format(x))

for n in range(30):
    L = fn(x)
    L.backward()
    # gradient descent update
    x.value = x.value - 0.1 * x.gradient
    # clear the gradients
    x.clear_gradient()
    
print('---- Converged Value ----')    
print('fn(x): {}'.format(L))
print('x: {}'.format(x))

'''
# Wolfram alpha minimum: 
# min{x^2 + 0.2 (x - 2)^4 + 2 x^3}≈1.5110 at x≈0.51489
'''

---- Initial Value ----
fn(x): < Variable value: 1.5110101217988139, gradient: 1.0 >
x: < Variable value: 2.0, gradient: 0.0 >
---- Converged Value ----
fn(x): < Variable value: 1.5110101217988139, gradient: 1.0 >
x: < Variable value: 0.5148854906394676, gradient: 0.0 >


'\n# Wolfram alpha minimum: \n# min{x^2 + 0.2 (x - 2)^4 + 2 x^3}≈1.5110 at x≈0.51489\n'

In [584]:
# dual number forward to backward auto-calc

# for factorial and trig functions
import math
    

def sin(dual_x):
    # force copy
    result = Dual(dual_x.value, dual_x.derivative)
    
    for i in range(8):
        factor = (-1)**(i+1)
        coeff = (i+1)*2 + 1
        result += factor * dual_x**coeff / math.factorial(coeff)
                  
    return result

def cos(dual_x):
    # return a Dual
    result = Dual(1.)
    
    for i in range(8):
        factor = (-1)**(i+1)
        coeff = (i+1)*2
        result += factor * dual_x**coeff / math.factorial(coeff)
                  
    return result

def exp(dual_x):
    # return a Dual
    result = Dual(1.)
    
    for i in range(100):
        coeff = i+1
        calc_exponent = Dual(1.)
        for n in range(coeff):
            calc_exponent *= dual_x
        result += calc_exponent * (1. / math.factorial(coeff))
                  
    return result

def log(dual_x):
    # return a Dual 
    # in future accept general type and call constructor
    result = Dual(0.)
    
    # TODOL implement comparison
    if (dual_x.value <= 0.):
        raise ArithmeticError
    if (dual_x.value > 1.5):
        scaling = power(Dual(1.5), Dual(-1.))
        return log(scaling * dual_x) + log(Dual(1.5))
    
    for i in range(100):
        n = (i+1)
        factor = (-1.)**(n-1)
        calc_exponent = Dual(1.)
        for _ in range(n):
            calc_exponent *= dual_x - 1.
        result +=  factor * calc_exponent * (1. / n)
                  
    return result

def power(x, y): 
    return exp(y*log(x))

def arith_div(x,y):
    # TODO: swap arguments 
    return x * power(y, Dual(-1.))
    

    
x = Dual(2., 0.)
y = Dual(1.1, 1.)
z = Dual(2., 0.)

print('dual numbers diff: sin(x + z*y)**0.5')
print(sin(x + z*y)**0.5)

print('power series expansion of sin')
print(sin(y))

print('actual value of sin (library): {} derivative: {}'.format(math.sin(y.value), math.cos(y.value)))
print()

print('power series expansion of exp')
print(exp(y))

print('actual value of exp (library): {} derivative: {}'.format(math.exp(y.value), math.exp(y.value)))


print('power series expansion of log')
print(log(y))

print('actual value of log (library): {} derivative: {}'.format(math.log(y.value),1. / (y.value)))

print('exploring power function')
x, y = Dual(4., 1.), Dual(1., 0.)
print(x**y)
print(power(x, y)) 

print('now with arith div')
print(arith_div(y, x))
print(y.value/x.value)
print((y.derivative * x.value - x.derivative * y.value) / x.value**2)

dual numbers diff: sin(x + z*y)**0.5
<Dual value: (5.716524457141041e-17+0.9335792917796556j), derivative: (-3.2153997568385185e-17+0.5251146304513603j)>
power series expansion of sin
<Dual value: 0.8912073600614354, derivative: 0.45359612142557815>
actual value of sin (library): 0.8912073600614354 derivative: 0.4535961214255773

power series expansion of exp
<Dual value: 3.004166023946434, derivative: 3.004166023946434>
actual value of exp (library): 3.0041660239464334 derivative: 3.0041660239464334
power series expansion of log
<Dual value: 0.09531017980432496, derivative: 0.9090909090909092>
actual value of log (library): 0.09531017980432493 derivative: 0.9090909090909091
exploring power function
<Dual value: 4.0, derivative: 1.0>
<Dual value: 4.0, derivative: 0.9999999999999996>
now with arith div
<Dual value: 0.2500000000000001, derivative: -0.062499999999999944>
0.25
-0.0625
